In [ ]:
import time
import numpy as np
import cv2
import os
from pynq import allocate
from pynq import Overlay

In [ ]:
overlay = Overlay('/home/xilinx/jupyter_notebooks/LPR/overlay/cnn.bit')
overlay.ip_dict
overlay.download()
conv = overlay.Conv_0
pool = overlay.pooling_0
print("ips are imported")

In [ ]:
def RunConv(conv,Kx,Ky,Sx,Sy,mode,relu_en,feature_in,W,bias,feature_out):
    conv.write(0x10,feature_in.shape[2])
    conv.write(0x18,feature_in.shape[0])
    conv.write(0x20,feature_in.shape[1])
    conv.write(0x28,feature_out.shape[2])
    conv.write(0x30,Kx)
    conv.write(0x38,Ky)
    conv.write(0x40,Sx)
    conv.write(0x48,Sy)
    conv.write(0x50,mode)
    conv.write(0x58,relu_en)
    conv.write(0x60,feature_in.physical_address)
    conv.write(0x68,W.physical_address)
    conv.write(0x70,bias.physical_address)
    conv.write(0x78,feature_out.physical_address)
    conv.write(0, (conv.read(0)&0x80)|0x01 )
    tp=conv.read(0)
    while not ((tp>>1)&0x1):
        tp=conv.read(0)

def RunPool(pool, Kx, Ky, Sc, Sx, Sy, mode, feature_in, feature_out):
    pool.write(0x10, feature_in.shape[2])
    pool.write(0x18, feature_in.shape[0])
    pool.write(0x20, feature_in.shape[1])
    pool.write(0x28, Kx)
    pool.write(0x30, Ky)
    pool.write(0x38, Sc)
    pool.write(0x40, Sx)
    pool.write(0x48, Sy)
    pool.write(0x50, mode)
    pool.write(0x58, feature_in.physical_address)
    pool.write(0x60, feature_out.physical_address)
    pool.write(0, (pool.read(0) & 0x80) | 0x01)
    while not ((pool.read(0) >> 1) & 0x1):
        pass;
    
print("drivers are coded")

In [ ]:
# Conv1
IN_WIDTH1 = 28
IN_HEIGHT1 = 28
IN_CH1 = 1

KERNEL_WIDTH1 = 5
KERNEL_HEIGHT1 = 5
X_STRIDE1 = 1
Y_STRIDE1 = 1

RELU_EN1 = 1
MODE1 = 1    # 0:VALID, 1:SAME
X_PADDING1 = 2
Y_PADDING1 = 2

OUT_CH1 = 6
OUT_WIDTH1 = int((IN_WIDTH1+2*X_PADDING1-KERNEL_WIDTH1)/X_STRIDE1+1)
OUT_HEIGHT1 = int((IN_HEIGHT1+2*Y_PADDING1-KERNEL_HEIGHT1)/Y_STRIDE1+1)

# Avgpool1
MODE_p1 = 0  # mode: 0:MEAN, 1:MAX
IN_WIDTH_p1 = OUT_WIDTH1
IN_HEIGHT_p1 = OUT_HEIGHT1
IN_CH_p1 = OUT_CH1

KERNEL_WIDTH_p1 = 2
KERNEL_HEIGHT_p1 = 2
C_STRIDE_p1 = 1
X_STRIDE_p1 = 2
Y_STRIDE_p1 = 2

OUT_CH_p1 = IN_CH_p1
OUT_WIDTH_p1 = int(IN_WIDTH_p1 / KERNEL_WIDTH_p1)
OUT_HEIGHT_p1 = int(IN_HEIGHT_p1 / KERNEL_HEIGHT_p1)

# Conv2
IN_WIDTH2 = OUT_WIDTH_p1
IN_HEIGHT2 = OUT_HEIGHT_p1
IN_CH2 = OUT_CH_p1

KERNEL_WIDTH2 = 5
KERNEL_HEIGHT2 = 5
X_STRIDE2 = 1
Y_STRIDE2 = 1

RELU_EN2 = 1
MODE2 = 0  # 0:VALID, 1:SAME
X_PADDING2 = 0
Y_PADDING2 = 0

OUT_CH2 = 16
OUT_WIDTH2 = int((IN_WIDTH2 + 2 * X_PADDING2 - KERNEL_WIDTH2) / X_STRIDE2 + 1)
OUT_HEIGHT2 = int((IN_HEIGHT2 + 2 * Y_PADDING2 - KERNEL_HEIGHT2) / Y_STRIDE2 + 1)

# Avgpool2
MODE_p2 = 0  # mode: 0:MEAN, 1:MAX
IN_WIDTH_p2 = OUT_WIDTH2
IN_HEIGHT_p2 = OUT_HEIGHT2
IN_CH_p2 = OUT_CH2

KERNEL_WIDTH_p2 = 2
KERNEL_HEIGHT_p2 = 2
C_STRIDE_p2 = 1
X_STRIDE_p2 = 2
Y_STRIDE_p2 = 2

OUT_CH_p2 = IN_CH_p2
OUT_WIDTH_p2 = int(IN_WIDTH_p1 / KERNEL_WIDTH_p1)
OUT_HEIGHT_p2 = int(IN_HEIGHT_p1 / KERNEL_HEIGHT_p1)

# Conv3
IN_WIDTH3 = OUT_WIDTH_p2
IN_HEIGHT3 = OUT_HEIGHT_p2
IN_CH3 = OUT_CH_p2

KERNEL_WIDTH3 = 5
KERNEL_HEIGHT3 = 5
X_STRIDE3 = 1
Y_STRIDE3 = 1

RELU_EN3 = 0
MODE3 = 0  # 0:VALID, 1:SAME
X_PADDING3 = 0
Y_PADDING3 = 0

OUT_CH3 = 120
OUT_WIDTH3 = int((IN_WIDTH3 + 2 * X_PADDING3 - KERNEL_WIDTH3) / X_STRIDE3 + 1)
OUT_HEIGHT3 = int((IN_HEIGHT3 + 2 * Y_PADDING3 - KERNEL_HEIGHT3) / Y_STRIDE3 + 1)

# dense
IN_WIDTH4 = OUT_WIDTH3
IN_HEIGHT4 = OUT_HEIGHT3
IN_CH4 = OUT_CH3

KERNEL_WIDTH4 = 1
KERNEL_HEIGHT4 = 1
X_STRIDE4 = 1
Y_STRIDE4 = 1

RELU_EN4 = 0
MODE4 = 0  # 0:VALID, 1:SAME
X_PADDING4 = 0
Y_PADDING4 = 0

OUT_CH4 = 84
OUT_WIDTH4 = int((IN_WIDTH4 + 2 * X_PADDING4 - KERNEL_WIDTH4) / X_STRIDE4 + 1)
OUT_HEIGHT4 = int((IN_HEIGHT4 + 2 * Y_PADDING4 - KERNEL_HEIGHT4) / Y_STRIDE4 + 1)

# output
IN_WIDTH5 = OUT_WIDTH4
IN_HEIGHT5 = OUT_HEIGHT4
IN_CH5 = OUT_CH4

KERNEL_WIDTH5 = 1
KERNEL_HEIGHT5 = 1
X_STRIDE5 = 1
Y_STRIDE5 = 1

RELU_EN5 = 0
MODE5 = 0  # 0:VALID, 1:SAME
X_PADDING5 = 0
Y_PADDING5 = 0

OUT_CH5 = 65
OUT_WIDTH5 = int((IN_WIDTH5 + 2 * X_PADDING5 - KERNEL_WIDTH5) / X_STRIDE5 + 1)
OUT_HEIGHT5 = int((IN_HEIGHT5 + 2 * Y_PADDING5 - KERNEL_HEIGHT5) / Y_STRIDE5 + 1)

print("parameters are defined")

In [ ]:
# input image
image = allocate(shape=(IN_HEIGHT1, IN_WIDTH1, IN_CH1), cacheable=0, dtype=np.float32)

# conv1
W_conv1 = allocate(shape=(KERNEL_HEIGHT1, KERNEL_WIDTH1, IN_CH1, OUT_CH1), cacheable=0, dtype=np.float32)
B_conv1 = allocate(shape=(OUT_CH1), cacheable=0, dtype=np.float32)
h_conv1 = allocate(shape=(OUT_HEIGHT1, OUT_WIDTH1, OUT_CH1), cacheable=0, dtype=np.float32)

# pool1
h_pool1 = allocate(shape=(OUT_HEIGHT_p1, OUT_WIDTH_p1, OUT_CH_p1), cacheable=0, dtype=np.float32)

# Conv2
W_conv2 = allocate(shape=(KERNEL_HEIGHT2, KERNEL_WIDTH2, IN_CH2, OUT_CH2), cacheable=0, dtype=np.float32)
B_conv2 = allocate(shape=(OUT_CH2), cacheable=0, dtype=np.float32)
h_conv2 = allocate(shape=(OUT_HEIGHT2, OUT_WIDTH2, OUT_CH2), cacheable=0, dtype=np.float32)

# Conv3
W_conv3 = allocate(shape=(KERNEL_HEIGHT3, KERNEL_WIDTH3, IN_CH3, OUT_CH3), cacheable=0, dtype=np.float32)
B_conv3 = allocate(shape=(OUT_CH3), cacheable=0, dtype=np.float32)
h_conv3 = allocate(shape=(OUT_HEIGHT3, OUT_WIDTH3, OUT_CH3), cacheable=0, dtype=np.float32)

# pool2
h_pool2 = allocate(shape=(OUT_HEIGHT_p2, OUT_WIDTH_p2, OUT_CH_p2), cacheable=0, dtype=np.float32)

# dense
W_conv4 = allocate(shape=(IN_CH4, OUT_CH4), cacheable=0, dtype=np.float32)
B_conv4 = allocate(shape=(OUT_CH4), cacheable=0, dtype=np.float32)
h_conv4 = allocate(shape=(OUT_HEIGHT4, OUT_WIDTH4, OUT_CH4), cacheable=0, dtype=np.float32)

# output
W_conv5 = allocate(shape=(IN_CH5, OUT_CH5), cacheable=0, dtype=np.float32)
B_conv5 = allocate(shape=(OUT_CH5), cacheable=0, dtype=np.float32)
h_conv5 = allocate(shape=(OUT_HEIGHT5, OUT_WIDTH5, OUT_CH5), cacheable=0, dtype=np.float32)

print("space are allocated")

In [ ]:
w_conv1 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/conv1_weight.npy')
for i in range(5):
    for j in range(5):
        for k in range(1):
            for l in range(6):
                W_conv1[i][j][k][l] = w_conv1[l][k][j][i]                
b_conv1 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/conv1_bias.npy')
for i in range(6):
    B_conv1[i] = b_conv1[i]
    
w_conv2 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/conv2_weight.npy')
for i in range(5):
    for j in range(5):
        for k in range(6):
            for l in range(16):
                W_conv2[i][j][k][l] = w_conv2[l][k][j][i]
b_conv2 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/conv2_bias.npy')
for i in range(16):
    B_conv2[i] = b_conv2[i]                
                
w_conv3 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/conv3_weight.npy')
for i in range(5):
    for j in range(5):
        for k in range(16):
            for l in range(120):
                W_conv3[i][j][k][l] = w_conv3[l][k][j][i]
b_conv3 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/conv3_bias.npy')
for i in range(120):
    B_conv3[i] = b_conv3[i]                
                
w_conv4 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/dense_weight.npy')
for i in range(120):
    for j in range(84):
        W_conv4[i][j] = w_conv4[j][i]
b_conv4 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/dense_bias.npy')
for i in range(84):
    B_conv4[i] = b_conv4[i]
    
w_conv5 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/output_weight.npy')
for i in range(84):
    for j in range(65):
        W_conv5[i][j] = w_conv5[j][i]
b_conv5 = np.load('/home/xilinx/jupyter_notebooks/LPR/weights/output_bias.npy')
for i in range(65):
    B_conv5[i] = b_conv5[i]
    
print("weights and bias are loaded")

In [ ]:
def licensePlateExtraction(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17))
    tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, kernel)

    y = cv2.Sobel(tophat, cv2.CV_16S, 1, 0)
    absY = cv2.convertScaleAbs(y)

    ret, binary = cv2.threshold(absY, 75, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 15))
    Open = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (41, 15))
    close = cv2.morphologyEx(Open, cv2.MORPH_CLOSE, kernel)

    kernel_x = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 7))
    kernel_y = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 11))

    erode_y = cv2.morphologyEx(close, cv2.MORPH_ERODE, kernel_y)
    dilate_y = cv2.morphologyEx(erode_y, cv2.MORPH_DILATE, kernel_y)

    dilate_x = cv2.morphologyEx(dilate_y, cv2.MORPH_DILATE, kernel_x)

    erode_x = cv2.morphologyEx(dilate_x, cv2.MORPH_ERODE, kernel_x)

    kernel_e = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 9))
    erode = cv2.morphologyEx(erode_x, cv2.MORPH_ERODE, kernel_e)

    kernel_d = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 11))
    dilate = cv2.morphologyEx(erode, cv2.MORPH_DILATE, kernel_d)

    img_copy = img.copy()

    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    i = 0
    for contour in contours:
        rect = cv2.boundingRect(contour)
        if rect[2] > rect[3] * 3 and rect[2] < rect[3] * 5:
            print(rect)
            img = img[(rect[1] - 5):(rect[1] + rect[3] + 5), (rect[0] - 5):(rect[0] + rect[2] + 5)]
            try:
                cv2.imwrite('/home/xilinx/jupyter_notebooks/LPR/plates/img%d.jpg' % (i), img)
                i += 1
            except:
                pass
    
def char_splitting(path):
    accuracy_num = 0
    img = cv2.imread(path)
    img1 = cv2.resize(img, (320, 100), interpolation=cv2.INTER_AREA)
    img2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img3 = cv2.bilateralFilter(img2, 11, 17, 17)
    img4 = cv2.Canny(img3, 50, 150)
    img5 = img4[10:90, 10:310]
    crop_img = img1[10:90, 10:310, :]

    contours, hierarchy = cv2.findContours(img5, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidate = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        if w*h < 500:
            continue
        if w * h > 4000:
            continue
        if h < 20:
            continue
        if w > 80:
            continue

        candidate.append([x, (x + w)])

    loc = np.zeros(300)

    for j in range(len(candidate)):
        x1 = candidate[j][0]
        x2 = candidate[j][1]
        loc[x1:x2] = 1

    start = []
    end = []

    if loc[0] == 1:
        start.append(0)

    for j in range(300-1):
        if loc[j] == 0 and loc[j+1] == 1:
            start.append(j)
        if loc[j] == 1 and loc[j+1] == 0:
            end.append(j)

    if loc[299] == 1:
        end.append(299)

    for i in range(7):
        cropped_img = crop_img[0:80, start[i]:end[i]]
        cv2.imwrite('/home/xilinx/jupyter_notebooks/LPR/plate_chars/' + str(i) + '.jpg', cropped_img)


In [ ]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B',
           'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P',
           'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
           '川', '鄂', '赣', '甘', '贵', '桂', '黑', '沪', '冀', '津', '京', '吉', '辽', '鲁', '蒙',
           '闽', '宁', '青', '琼', '陕', '苏', '晋', '皖', '湘', '新', '豫', '渝', '粤', '云', '藏', '浙']

lprResult = []

def Lpr(imagePath):
    image1 = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    image1 = cv2.resize(image1, (28, 28))
    for i in range(IN_HEIGHT1):
        for j in range(IN_WIDTH1):
            for k in range(IN_CH1):
                image[i][j][k] = (255 - image1[i][j]) / 255
    RunConv(conv, KERNEL_WIDTH1, KERNEL_HEIGHT1, X_STRIDE1, Y_STRIDE1, MODE1, RELU_EN1, image, W_conv1, B_conv1,
            h_conv1)
    RunPool(pool, KERNEL_WIDTH_p1, KERNEL_HEIGHT_p1, C_STRIDE_p1, X_STRIDE_p1, Y_STRIDE_p1, MODE_p1, h_conv1, h_pool1)
    RunConv(conv, KERNEL_WIDTH2, KERNEL_HEIGHT2, X_STRIDE2, Y_STRIDE2, MODE2, RELU_EN2, h_pool1, W_conv2, B_conv2,
            h_conv2)
    RunPool(pool, KERNEL_WIDTH_p2, KERNEL_HEIGHT_p2, C_STRIDE_p2, X_STRIDE_p2, Y_STRIDE_p2, MODE_p2, h_conv2, h_pool2)
    RunConv(conv, KERNEL_WIDTH3, KERNEL_HEIGHT3, X_STRIDE3, Y_STRIDE3, MODE3, RELU_EN3, h_pool2, W_conv3, B_conv3,
            h_conv3)
    RunConv(conv, KERNEL_WIDTH4, KERNEL_HEIGHT4, X_STRIDE4, Y_STRIDE4, MODE4, RELU_EN4, h_conv3, W_conv4, B_conv4,
            h_conv4)
    RunConv(conv, KERNEL_WIDTH5, KERNEL_HEIGHT5, X_STRIDE5, Y_STRIDE5, MODE5, RELU_EN5, h_conv4, W_conv5, B_conv5,
            h_conv5)
    MAX = -9999999999999999
    for i in range(OUT_CH5):
        if h_conv5[0][0][i] > MAX:
            MAX = h_conv5[0][0][i]
            index = i
    lprResult.append(classes[index])

In [ ]:
licensePlateExtraction('/home/xilinx/jupyter_notebooks/LPR/cars/car01.png')
char_splitting('/home/xilinx/jupyter_notebooks/LPR/plates/img0.jpg')
directory = '/home/xilinx/jupyter_notebooks/LPR/plate_chars'

for root, dirs, files in os.walk(directory):
    for file in files:
        path = os.path.join(root, file)
        Lpr(path)

print("the result of license plate recognition is " + "".join(lprResult))